In [ ]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)

import ipywidgets

import datetime
import math
import bisect
import time
from os import listdir
from os.path import isfile, join

In [ ]:
daydatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_day'''
files = [f for f in listdir(daydatadir) if isfile(join(daydatadir, f))]
instruments = [f[0:-4] for f in files]

colnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
daypricedata = {}
for f in instruments:
    df = pd.read_csv(join(daydatadir, f+'.txt'), names=colnames)
    df['Date'] = df['Date'].map(lambda x: datetime.date(int(x[6:10]), int(x[0:2]), int(x[3:5])))
    df['Gap'] = np.insert(np.array(df['Open'][1:])-np.array(df['Close'][:-1]),0,0)
    df = df.set_index('Date')
    daypricedata[f] = df

In [ ]:
daypricedata['A']['Open'].iloc[0]

In [ ]:
start_date = datetime.date(2018,1,1)
end_date = datetime.date(2018,7,1)
num_instruments = 1
max_gap_instruments = {}

for ii in range((end_date - start_date).days):
    date = start_date + datetime.timedelta(days=ii)
    max_gap = 0
    max_gap_instrument = None
    for instrument in instruments:
        if date in daypricedata[instrument].index:
            gap = abs(daypricedata[instrument].loc[date]['Gap'])/daypricedata[instrument].loc[date]['Open']
            if gap > max_gap:
                max_gap = gap
                max_gap_instrument = instrument
    max_gap_instruments[date] = max_gap_instrument
    print(date, max_gap_instrument, max_gap)
            
    

In [ ]:
def gap_trading_strategy(portfolio, data_dict, param_dict):
    minutedatadir = r'''C:\ProgramData\Kibot Agent\Data\SP500_1'''
    minutecolnames = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    
    max_gap_instruments = data_dict['max_gap_instruments']
    
    for ii in range((end_date - start_date).days):
        date = start_date + datetime.timedelta(days=ii)
        
        instrument = max_gap_instruments[date]
        if instrument:
            try:
                pricedata = pd.read_csv(join(minutedatadir, instrument+'.txt'), names=minutecolnames)
                pricedata['DateTime'] = (pricedata['Date']+pricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
                pricedata = pricedata[[(dt >= datetime.datetime(date.year, date.month, date.day, 9, 30, 0)) and (dt <= datetime.datetime(date.year, date.month, date.day, 16, 0, 0)) for dt in pricedata['DateTime']]].copy()

                portfolio.positions[instrument] = 0
                print(date, instrument)
            except:
                continue
            
            if not pricedata.index.empty and len(pricedata) > 1:
                open_price = pricedata['Open'].iloc[1]
                for index in range(len(pricedata)):
                    row = pricedata.iloc[index]

                    if index == len(pricedata)-1:
                        if portfolio.positions[instrument] < 0:
                            portfolio.buy(instrument, -portfolio.positions[instrument], row)
                        elif portfolio.positions[instrument] > 0:
                            portfolio.sell(instrument, portfolio.positions[instrument], row)
                    elif row['Close'] > open_price:
                        if portfolio.positions[instrument] <= 0:
                            quantity = np.floor(10000/row['Close'])
                            portfolio.buy(instrument, -portfolio.positions[instrument] + quantity, row)
                    elif row['Close'] < open_price:
                        if portfolio.positions[instrument] >= 0:
                            quantity = np.floor(10000/row['Close'])
                            portfolio.sell(instrument, portfolio.positions[instrument] + quantity, row)

                    portfolio.update(instrument, row)
    
    portfolio.portfolio_history = pd.DataFrame(portfolio.portfolio_history)
    portfolio.transaction_history = pd.DataFrame(portfolio.transaction_history)                

In [ ]:
class Portfolio:
    def __init__(self, starting_cash=10000, commission=0):
        self.cash = starting_cash
        self.positions = {}
        self.commission = commission

        self.ask_price_key = 'Close'
        self.bid_price_key = 'Close'
        
        self.net_value = 0
        self.portfolio_history = []
        self.transaction_history = []
        
    def update(self, instrument, row):
        price = row[self.bid_price_key]
        self.net_value = self.cash + self.positions[instrument] * price
        self.portfolio_history.append(
            {
                "DateTime": row['DateTime'],
                "Cash": self.cash,
                "Position": self.positions,
                "Net Value": self.net_value
            }
        )
        
    def buy(self, instrument, quantity, row):
        price = row[self.ask_price_key]
        cost = quantity*price + self.commission
        self.cash -= cost

        if not instrument in self.positions.keys():
            positions[instrument] = 0
        self.positions[instrument] += quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": row['DateTime'],
                "Instrument" : instrument,
                "Price": price,
                "Quantity": quantity
            }
        )
        
    def sell(self, instrument, quantity, row):
        price = row[self.bid_price_key]
        self.cash += quantity*price - self.commission

        if not instrument in self.positions.keys():
            positions[instrument] = 0
        self.positions[instrument] -= quantity

        self.cash = round(self.cash*100)/100

        self.transaction_history.append(
            {
                "DateTime": row['DateTime'],
                "Instrument" : instrument,
                "Price": price,
                "Quantity": -quantity
            }
        )

In [ ]:
data_dict = {'max_gap_instruments': max_gap_instruments}
portfolio = Portfolio(starting_cash = 10000, commission=5)

param_dict = {}
gap_trading_strategy(portfolio, data_dict, param_dict)

In [ ]:
class ProfitCharts():
    def __init__(self, portfolio):
        self.portfolio = portfolio
        cash = portfolio.portfolio_history['Cash']
        net_value = portfolio.portfolio_history['Net Value']
        
        x_vals = [dt.strftime("%y/%m/%d, %H:%M:%S") for dt in portfolio.portfolio_history['DateTime']]
        tick_vals = [ii for ii in range(len(x_vals)) if 
                                 portfolio.portfolio_history['DateTime'][ii].time() == datetime.time(9, 30)]
        xaxisdatetimes = [portfolio.portfolio_history['DateTime'][ii] for ii in tick_vals]
        tick_text = [dt.strftime(' %y/%m/%d ') if dt.time() == datetime.time(9, 30)
                                 else dt.strftime(' %H:%M ') for dt in xaxisdatetimes]

        data1 = [ dict(
                type = 'scatter',
                x = x_vals,
                y = cash,
                name = 'Cash'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value - cash,
                name = 'Position'),

                dict(
                type = 'scatter',
                x = x_vals,
                y = net_value,
                name = 'Net Value')
               ]
        
        layout1 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Value'
                    )
                ))
        
        self.chart1 = go.FigureWidget( data=data1, layout=layout1)
        
        data2 = [
                dict(
                type = 'scatter',
                x = x_vals,
                y = [nv - net_value[0] for nv in net_value],
                name = 'Profit')
               ]
        
        layout2 = dict(
                title = 'Profit Chart',
                xaxis = dict(
                    title = dict(
                        text = 'DateTime'
                    ),
                    type = 'category', 
                    categoryorder = 'category ascending',
                    tickvals = tick_vals,
                    ticktext = tick_text
                ),
                yaxis = dict(
                    title = dict(
                        text = 'Profit'
                    )
                ))
        
        self.chart2 = go.FigureWidget( data=data2, layout=layout2)        
        # DEFINE DISPLAY CONFIGURATION
        self.display = ipywidgets.VBox([self.chart1, self.chart2])

In [ ]:
portfolio.transaction_history

In [ ]:
profit_charts = ProfitCharts(portfolio)
profit_charts.display